# 1. Radom Forest

In [1]:
from sklearn.ensemble import RandomForestRegressor
import time
import pickle
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
import shap
import gc
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

pd.set_option('display.max_rows', 160)
pd.set_option('display.max_columns', 160)
pd.set_option('display.max_colwidth', 30)

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_pickle('df_completeMy.pkl')
#del data['subtype']

X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)
#free memory
del data
gc.collect();

In [5]:
# params = {
#     'n_estimators': [10, 20, 40, 80, 100],
#     'criterion': ['gini', 'entropy'],
#     'max_depth': [2, 4, 6, 8, 10],
#     'bootstrap': [True, False],
# #     'min_samples_leaf':[10, 20, 40, 100],
# #     'min_samples_split':[10, 20, 40, 100],
# #     'max_features':['auto', 'sqrt', 'log2'],
# }
# estimator=RandomForestClassifier()
# rf_model = GridSearchCV(estimator=estimator, param_grid=params
#                             , scoring='neg_mean_squared_error', cv=3, verbose=1)
rf_model = RandomForestRegressor(n_estimators=60,
                          min_samples_leaf=15,
                          n_jobs=-1,
                          random_state=0)
rf_model.fit(X_train, Y_train)

RandomForestRegressor(min_samples_leaf=15, n_estimators=60, n_jobs=-1,
                      random_state=0)

In [7]:
submission = pd.read_csv('./sample_submission.csv')
submission['item_cnt_month'] = rf_model.predict(X_test).clip(0,20)
submission[['ID', 'item_cnt_month']].to_csv('initial_rf_submission.csv', index=False)